In [2]:
import os
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
import gffutils
#from Bio import SeqIO
#import plotly.graph_objects as pygo

os.path.normpath("C:\\Users\\BMH_work\\github\\yeast_esr_expression_analysis\\expression_data\\ortholog_files_regev\\SGD_features.tab")


In [65]:
## Id conversions

#Read in orf/name file and make it a dictionary
# SC_features_fname = os.path.normpath(data_processing_dir + "\ortholog_files\\SGD_features.tab")

SC_features_fname = os.path.normpath("C:\\Users\\BMH_work\\github\\yeast_esr_expression_analysis\\expression_data\\ortholog_files_regev\\SGD_features.tab")

SC_features = pd.read_csv(SC_features_fname, sep = '\t', header=None)
SC_orfs = SC_features.groupby(1).get_group('ORF')

#Makes a dictionary to look up orfs by gene names.  This won't include all orfs - those without names had NaN in column 4 so 
#are presumably left out. 
#SC_orfs_lookup = dict(zip(SC_orfs[4], SC_orfs[3]))  #This dictionary might be messed up - not one to one.  
SC_genename_lookup = dict(zip(SC_orfs[3], SC_orfs[4]))
#SC_features_lookup = dict(zip(SC_orfs[3], SC_orfs[15]))

def SC_common_name_lookup(gene, SC_genename_lookup):
    try:  
        SC_common_name = SC_genename_lookup[gene]
        if isinstance(SC_common_name,float):
            if np.isnan(SC_common_name):
                SC_common_name = gene
            else: 
                print('float but not nan uh oh!')
        else: 
            SC_common_name = SC_genename_lookup[gene]
    except KeyError: 
        SC_common_name = gene
    
    return SC_common_name

In [83]:
#Create db
genome_dir = os.path.normpath('C:/Users/BMH_work/Google Drive/Crick_LMS/external_data/genomes/scer_ref')

sc_pel_max_gtf_fname = genome_dir + os.sep + 'saccharomyces_cerevisiae_R64-2-1_20150113_UTRs_pelechano_max.gff' 
sc_pel_max_gtf_db_fname = genome_dir + os.sep + 'saccharomyces_cerevisiae_R64-2-1_20150113_UTRs_pelechano_max.db'
#sc_pel_max_gtf_db= gffutils.create_db(sc_pel_max_gtf_fname, sc_pel_max_gtf_fname)

sc_pel_max_gtf_db = gffutils.FeatureDB(sc_pel_max_gtf_db_fname) 

fname_out_unsorted = genome_dir + os.sep + 'saccharomyces_cerevisiae_R64-2-1_20150113_UTRs_pelechano_cellranger_4FP_unsorted.gtf'
fname_out = genome_dir + os.sep + 'saccharomyces_cerevisiae_R64-2-1_20150113_UTRs_pelechano_cellranger_4FP.gtf' 

In [19]:
feature = sc_pel_max_gtf_db['YPR195C_CDS_3pUTR.long']

In [ ]:
#Iterate through CDS and make gff
#Iterate through gene and make gff
#Iterate through 3pUTR->Exon




In [85]:
with open(fname_out_unsorted, 'w') as fout: 
    for feature in sc_pel_max_gtf_db.features_of_type('CDS_3pUTR'):
        transcript_id = feature.attributes['ID'][0]
        gene_id = feature.attributes['Parent'][0]
        gene_name = SC_common_name_lookup(gene_id, SC_genename_lookup)
        
        attributes = 'transcript_id "{}"; gene_id "{}"; gene_name "{}";'.format(transcript_id, gene_id, gene_name)
        
        line_items= [feature.seqid,
        feature.source,
        'exon',
        str(feature.start),
        str(feature.end),
        feature.score, 
        feature.strand,
        str(feature.frame), 
        attributes]

        line = '\t'.join(line_items) + '\n'
        fout.write(line)
    
    for feature in sc_pel_max_gtf_db.features_of_type('CDS'):
        transcript_id = feature.attributes['ID'][0]
        gene_id = feature.attributes['Parent'][0].split('_')[0]
        gene_name = SC_common_name_lookup(gene_id, SC_genename_lookup)
        
        attributes = 'transcript_id "{}"; gene_id "{}"; gene_name "{}";'.format(transcript_id, gene_id, gene_name)
        
        line_items= [feature.seqid,
        feature.source,
        feature.featuretype,
        str(feature.start),
        str(feature.end),
        feature.score, 
        feature.strand,
        str(feature.frame), 
        attributes]

        line = '\t'.join(line_items) + '\n'
        fout.write(line)

    for feature in sc_pel_max_gtf_db.features_of_type('gene'):
        transcript_id = feature.attributes['ID'][0]
        gene_id = transcript_id
        gene_name = SC_common_name_lookup(gene_id, SC_genename_lookup)
        
        attributes = 'transcript_id "{}"; gene_id "{}"; gene_name "{}";'.format(transcript_id, gene_id, gene_name)
        
        line_items= [feature.seqid,
        feature.source,
        feature.featuretype,
        str(feature.start),
        str(feature.end),
        feature.score, 
        feature.strand,
        str(feature.frame), 
        attributes]

        line = '\t'.join(line_items) + '\n'
        fout.write(line)

In [94]:
gtf_unsorted_db.conn.close()

In [95]:
#Open new gff
fname_out_unsorted_4FP = genome_dir + os.sep + 'saccharomyces_cerevisiae_R64-2-1_20150113_UTRs_pelechano_cellranger_4FP_unsorted_4FP.gtf'
gtf_unsorted_db_fname = genome_dir + os.sep + 'saccharomyces_cerevisiae_R64-2-1_20150113_UTRs_pelechano_cellranger_4FP_unsorted_4FP.db'
gtf_unsorted_db= gffutils.create_db(fname_out_unsorted_4FP, gtf_unsorted_db_fname, force =True)
#sc_pel_max_gtf_db = gffutils.FeatureDB(sc_pel_max_gtf_fname) 

#Sort gff

#convert to gtf



In [96]:
with open(fname_out, 'w') as outfile:
    #outfile.write('\n')
    for feature in gtf_unsorted_db.all_features(order_by = ('seqid','start','featuretype')):
         print(feature,file=outfile)

In [ ]:
# #Add 4FP features

# cds_utr3p = gffutils.Feature(seqid=seqid, 
#                              source = 'Heineike_2020',
#                              featuretype = 'CDS_3pUTR',
#                              start = coords[0],
#                              end = coords[1],
#                              strand = strand,
#                              frame = '.', 
#                              attributes = {'ID': [genename + '_CDS_3pUTR.long'], 
#                                            'Parent': [genename], 
#                                            'utr': [utr_flag]  
#                                           }
#                                    )

In [ ]:
# merged_db = gffutils.create_db(merged_fn, dbfn=genome_assy_dir + os.sep + sc_ref_base + '_tifs.db', 
#                                id_spec=('ID', 'Name'), 
#                                merge_strategy='error', 
#                                force=True,  #makes new database
#                                #keep_order=True, see no reason to keep this
#                                force_gff = True)

In [97]:
# transcript_id = feature.attributes['ID'][0]
# gene_id = feature.attributes['Parent'][0]
# gene_name = SC_common_name_lookup(feature.attributes['Parent'][0], SC_genename_lookup)

# attributes = 'transcript_id "{}"; gene_id "{}"; gene_name "{}";'.format(transcript_id, gene_id, gene_name)

# line_items= [feature.seqid,
# feature.source,
# feature.featuretype,
# str(feature.start),
# str(feature.end),
# feature.score, 
# feature.strand,
# str(feature.frame), 
# attributes]



# '\t'.join(line_items) + '\n'
# #feature.attributes